In [ ]:
# 导入所需的模块
import random
import numpy as np
import math



# 定义一个函数，用于实现iBAT方法.
###############构建异常概率函数
###############
###########
def outlierscores(h,m):
    # 假设 m 为样本个数,应该是int
    #h 是深度，深度越小，异常概率越大,应该是int
    a = 2 * math.log(m - 1) + math.e - 2 * ((m - 1) / m)
    result = 2 ** (-h / a)
    return(result)


def iBAT(t, T, m, k):
    # 输入参数：
    # t - 测试轨迹
    # T - 要与之分离的轨迹集合
    # m - 运行试验的次数
    # k - 子样本的大小
    # 输出参数：
    # s - 异常分数，表示测试轨迹t与轨迹集合T的差异程度

    # 初始化一个长度为m的零向量n，用于记录每次试验中与测试轨迹t相似的轨迹的数量
    n = [0] * m

    # 对于每次试验i
    for i in range(m):
        # 从轨迹集合T中随机抽取k个轨迹作为子集T'
        T_prime = random.sample(T, k) #T需要是列表
        #print(f'第{i}次实验','选择的子集T为:',T_prime)
        # 重复以下步骤，直到子集T'为空
        while T_prime:
            # 将n[i]加一，表示当前子集中有一个轨迹与测试轨迹t相似
            n[i] += 1
            if n[i] <=10:
                
                    # 从测试轨迹t中随机选择一个单元格p，表示测试轨迹t经过的一个位置
                #p = random.choice(t)
                p = random.sample(t, 20)
                #print(f'第{n[i]}次选择','随机选择的单元格为:',p)
                # 从子集T'中筛选出包含单元格p的轨迹，作为新的子集T'，表示这些轨迹与测试轨迹t有共同的位置
                #T_prime = [traj for traj in T_prime if p in traj]
                T_prime = [traj for traj in T_prime if any(cell in traj for cell in p)]
                #print('选择的子集T中和检测的轨迹共同有此单元格的轨迹为:',T_prime)
            else:
                break
    a = []
    for i in range(m):
        o = outlierscores(n[i]-1,len(T))
        a.append(o)
    return np.mean(a)

In [ ]:
###############此为测试算法可用
###########
###########
import pandas as pd

t1 = [("A1",), ("A2",), ("A3",), ("A4",), ("A5",), ("B5",), ("C5",), ("D5",), ("E5",)]
t6 = [("A1",), ("A2",), ("A3",), ("A4",), ("A5",), ("B5",), ("C5",), ("D5",), ("D4",), ("E4",), ("E5",)]
t2 = [("A1",), ("B1",), ("C1",), ("D1",), ("E1",), ("E2",), ("E3",), ("E4",), ("E5",)]
t3 = [("A1",), ("A2",), ("A3",), ("B3",), ("C3",), ("D3",), ("E3",), ("E4",), ("E5",)]
t4 = [("A1",), ("A2",), ("B2",), ("C3",), ("C4",), ("D4",), ("D5",), ("E5",)]
t5 = [("A1",), ("B1",), ("C1",), ("D1",), ("E1",), ("F1",), ("G1",), ("G2",), ("F2",), ("E2",), ("E3",), ("E4",), ("E5",)]

# 合并所有的列表
all_lists = [t1, t6, t2, t3, t4, t5]

# 创建DataFrame
df = pd.DataFrame({'t': all_lists})
df0 = df
###########
i = 6#第几条轨迹
#########
b = i-1
t = df.iloc[b].values[0] # 测试轨迹
# 删除第5行
df = df.drop(b)
# 重置索引
df = df.reset_index(drop=True)
# 初始化一个空列表
T = []

# 迭代DataFrame的每一行，并将其添加到列表中
for index, row in df.iterrows():
    T.append(row['t'])
###########
m = 10 # 运行试验的次数
k = 2 # 子样本的大小
##########
iBAT(t, T, m, k)  #这是异常的

In [ ]:
df = df0
###########
i = 1#第几条轨迹
#########
b = i-1
t = df.iloc[b].values[0] # 测试轨迹
# 删除第5行
df = df.drop(b)
# 重置索引
df = df.reset_index(drop=True)
# 初始化一个空列表
T = []

# 迭代DataFrame的每一行，并将其添加到列表中
for index, row in df.iterrows():
    T.append(row['t'])
###########
m = 10 # 运行试验的次数
k = 2 # 子样本的大小
##########
iBAT(t, T, m, k)#这是正常的


In [ ]:
#在这里确定k和m
k = 80 # 子样本的大小
for n in range(10000):
    if (1-(k/439))**n < 0.0001:
        print(n)
        m = n # 运行试验的次数
        break
        

In [ ]:
###############主体代码
############
############
#取用部分数据测试参数
ib = [] #用来记录异常概率
ab = [] #用来记录对应的标签
for i in range(1, len(df_filtered)+1):
    #i为第几条轨迹
    #########
    b = i-1
    AbnormalState = df1[df1['GlobalID'] == df_filtered.iloc[b].values[0]]['AbnormalState'].values
    t = df_filtered.iloc[b].values[1] # 测试轨迹
    # 删除第b行
    df_filtered = df_filtered.drop(b)
    # 重置索引
    df_filtered = df_filtered.reset_index(drop=True)
    # 初始化一个空列表
    T = []

    # 迭代DataFrame的每一行，并将其添加到列表中
    for index, row in df_filtered.iterrows():
        T.append(row['GPSPOINTS'])
    ###########
    
    ##########
    if i%10 == 0:
        print(f'第{i}条轨迹')
    ib.append(iBAT(t, T, m, k))
    ab.append(AbnormalState)
    df_filtered = df2
# 创建DataFrame
df_result = pd.DataFrame({'ib': ib, 'ab': ab})

# 按照 'ib' 列进行排序，并获取最大的20个值
top_values = df_result.sort_values(by='ib', ascending=False).head(40)
#print(top_values)

# 计算 'ab' 列为[-1]的行数
count_ab_minus_one = top_values[top_values['ab'] == -1].shape[0]

# 计算总行数
total_rows = top_values.shape[0]

# 计算百分比
percentage_ab_minus_one = (count_ab_minus_one / total_rows) * 100

# 打印结果
print(f"百分比: {percentage_ab_minus_one}%")

In [ ]:
# 按照 'ib' 列进行排序，并获取最大的20个值
top_values = df_result.sort_values(by='ib', ascending=False).head(50)
#print(top_values)

# 计算 'ab' 列为[-1]的行数
count_ab_minus_one = top_values[top_values['ab'] == -1].shape[0]

# 计算总行数
total_rows = top_values.shape[0]

# 计算百分比
percentage_ab_minus_one = (count_ab_minus_one / total_rows) * 100

# 打印结果
print(f"百分比: {percentage_ab_minus_one}%")